# Invasive Devices
Feature Engineering: Invasive Devices



Load libraries and setup environment

In [1]:
# Import libraries
from datetime import timedelta
import os

import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from IPython.display import display, HTML, Image
%matplotlib inline

plt.style.use('ggplot')
plt.rcParams.update({'font.size': 20})

MIMIC BigQuery access

In [2]:
# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

# authenticate
auth.authenticate_user()

# MIMIC project ID
project_id = "ml-health-395010"

MIMIC IV setup (MIMIC IV version 2.2)

In [3]:
# CODE TAKEN FROM TUTORIAL 3
# accessed at: 14-08-2023

# Set up environment variables
if project_id == 'CHANGE-ME':
  raise ValueError('You must change project_id to your GCP project.')
os.environ["GOOGLE_CLOUD_PROJECT"] = project_id

# Read data from BigQuery into pandas dataframes.
def run_query(query, project_id=project_id):
  return pd.io.gbq.read_gbq(
      query,
      project_id=project_id,
      dialect='standard')

# set the dataset
# if you want to use the demo, change this to mimic_demo
dataset = 'mimiciv'


## Central Line-Associated Bloodstream Infection (CLABSI)

In [7]:
clabsi = """
SELECT
    mv.stay_id,
    -- mv.itemid,
    CASE
        WHEN di.label IN ('Arterial Line', 'A-Line') THEN 'Arterial'
        WHEN di.label IN ('CCO PA Line', 'CCO PAC') THEN 'Continuous Cardiac Output PA'
        WHEN di.label IN ('Dialysis Catheter', 'Dialysis Line') THEN 'Dialysis'
        WHEN di.label IN ('Hickman', 'Tunneled (Hickman) Line') THEN 'Hickman'
        WHEN di.label IN ('IABP', 'IABP line') THEN 'IABP'
        WHEN di.label IN ('Multi Lumen', 'Multi-lumen') THEN 'Multi Lumen'
        WHEN di.label IN ('PA Catheter', 'PA line') THEN 'PA'
        WHEN di.label IN ('PICC Line', 'PICC line') THEN 'PICC'
        WHEN di.label IN ('Pre-Sep Catheter', 'Presep Catheter') THEN 'Pre-Sep'
        WHEN di.label IN ('Trauma Line', 'Trauma line') THEN 'Trauma'
        WHEN di.label IN ('Triple Introducer', 'TripleIntroducer') THEN 'Triple Introducer'
        WHEN di.label IN ('Portacath', 'Indwelling Port (PortaCath)') THEN 'Portacath'
        ELSE di.label
    END AS type,
    mv.starttime,
    mv.endtime,
    CASE
        WHEN mv.location IN ('Left Antecub', 'Left Antecube') THEN 'Left Antecube'
        WHEN mv.location IN ('Left Axilla', 'Left Axilla.') THEN 'Left Axilla'
        WHEN mv.location IN ('Left Brachial', 'Left Brachial.') THEN 'Left Brachial'
        WHEN mv.location IN ('Left Femoral', 'Left Femoral.') THEN 'Left Femoral'
        WHEN mv.location IN ('Right Antecub', 'Right Antecube') THEN 'Right Antecube'
        WHEN mv.location IN ('Right Axilla', 'Right Axilla.') THEN 'Right Axilla'
        WHEN mv.location IN ('Right Brachial', 'Right Brachial.') THEN 'Right Brachial'
        WHEN mv.location IN ('Right Femoral', 'Right Femoral.') THEN 'Right Femoral'
        ELSE mv.location
    END AS value -- location
FROM `physionet-data.mimiciv_icu.procedureevents` mv
INNER JOIN `physionet-data.mimiciv_icu.d_items` di
ON mv.itemid = di.itemid
WHERE mv.itemid IN
(
    227719 -- AVA Line
    , 225752 -- Arterial Line
    , 224269 -- CCO PAC
    , 224267 -- Cordis/Introducer
    , 224270 -- Dialysis Catheter
    , 224272 -- IABP line
    , 226124 -- ICP Catheter
    , 228169 -- Impella Line
    , 225202 -- Indwelling Port (PortaCath)
    , 228286 -- Intraosseous Device
    , 225204 -- Midline
    , 224263 -- Multi Lumen
    , 224560 -- PA Catheter
    , 224264 -- PICC Line
    , 225203 -- Pheresis Catheter
    , 224273 -- Presep Catheter
    , 225789 -- Sheath
    , 225761 -- Sheath Insertion
    , 228201 -- Tandem Heart Access Line
    , 228202 -- Tandem Heart Return Line
    , 224268 -- Trauma line
    , 225199 -- Triple Introducer
    , 225315 -- Tunneled (Hickman) Line
    , 225205 -- RIC
)
AND mv.location is not null   --- !!!!! COMMENT THIS LINE WHEN EXTRATING FEATURE
LIMIT 1000
;
"""

In [8]:
run_query(clabsi)

,stay_id,type,starttime,endtime,value
0,38934988,ICP Catheter,2179-07-14 15:33:00,2179-07-15 11:30:00,Lumbar
1,38427719,ICP Catheter,2156-04-13 23:19:00,2156-04-18 17:00:00,Lumbar
2,37277638,ICP Catheter,2189-10-29 16:16:00,2189-10-31 14:00:00,Lumbar
3,33018681,ICP Catheter,2173-10-05 19:30:00,2173-10-07 08:05:00,Lumbar
4,32944560,ICP Catheter,2171-04-04 18:13:00,2171-04-09 17:15:00,Lumbar
...,...,...,...,...,...
995,33589102,Multi Lumen,2128-08-24 02:11:00,2128-08-27 15:01:00,Left IJ
996,38377313,Multi Lumen,2179-10-29 21:30:00,2179-11-02 12:00:00,Left IJ
997,38080928,Multi Lumen,2168-08-05 04:45:00,2168-08-07 00:32:00,Left IJ
998,39367199,Multi Lumen,2173-05-25 16:01:00,2173-05-26 15:01:00,Left IJ


Note:   
- value is location

## Ventilator-Associated Pneumonia (VAP)

In [9]:
vap = """
SELECT
    tm.stay_id,
    CASE
        WHEN ventilator_mode is not null THEN ventilator_mode
        WHEN ventilator_mode_hamilton is not null THEN ventilator_mode_hamilton
        ELSE null
    END as type,
    MIN(tm.charttime) as starttime,
    MAX(tm.charttime) as endtime,
    o2_delivery_device_1 as value
FROM
  (
    SELECT stay_id, charttime
    FROM `physionet-data.mimiciv_derived.ventilator_setting`
    UNION DISTINCT
    SELECT stay_id, charttime
    FROM `physionet-data.mimiciv_derived.oxygen_delivery`
  ) as tm
  LEFT JOIN `physionet-data.mimiciv_derived.ventilator_setting` vs
      ON tm.stay_id = vs.stay_id
      AND tm.charttime = vs.charttime
  LEFT JOIN `physionet-data.mimiciv_derived.oxygen_delivery` od
      ON tm.stay_id = od.stay_id
      AND tm.charttime = od.charttime
WHERE
    o2_delivery_device_1 IN
    (
        'Endotracheal tube'
    )
    OR ventilator_mode IN
    (
        '(S) CMV',
        'APRV',
        'APRV/Biphasic+ApnPress',
        'APRV/Biphasic+ApnVol',
        'APV (cmv)',
        'Ambient',
        'Apnea Ventilation',
        'CMV',
        'CMV/ASSIST',
        'CMV/ASSIST/AutoFlow',
        'CMV/AutoFlow',
        'CPAP/PPS',
        'CPAP/PSV+Apn TCPL',
        'CPAP/PSV+ApnPres',
        'CPAP/PSV+ApnVol',
        'MMV',
        'MMV/AutoFlow',
        'MMV/PSV',
        'MMV/PSV/AutoFlow',
        'P-CMV',
        'PCV+',
        'PCV+/PSV',
        'PCV+Assist',
        'PRES/AC',
        'PRVC/AC',
        'PRVC/SIMV',
        'PSV/SBT',
        'SIMV',
        'SIMV/AutoFlow',
        'SIMV/PRES',
        'SIMV/PSV',
        'SIMV/PSV/AutoFlow',
        'SIMV/VOL',
        'SYNCHRON MASTER',
        'SYNCHRON SLAVE',
        'VOL/AC'
    )
    OR ventilator_mode_hamilton IN
    (
        'APRV',
        'APV (cmv)',
        'Ambient',
        '(S) CMV',
        'P-CMV',
        'SIMV',
        'APV (simv)',
        'P-SIMV',
        'VS',
        'ASV'
    )
GROUP BY
    tm.stay_id, type, o2_delivery_device_1
HAVING
    -- type is not null
    -- AND o2_delivery_device_1 is not null
    MIN(tm.charttime) != MAX(tm.charttime)
LIMIT 1000
;
"""

In [10]:
run_query(vap)

,stay_id,type,starttime,endtime,value
0,36283909,CMV/ASSIST/AutoFlow,2142-10-06 18:00:00,2142-10-06 19:00:00,None
1,30532359,None,2175-04-11 22:59:00,2175-04-12 01:10:00,Endotracheal tube
2,38980177,Ambient,2145-04-15 08:00:00,2145-04-15 18:00:00,Trach mask
3,31817310,None,2142-09-06 16:50:00,2142-09-06 17:00:00,Endotracheal tube
4,36796263,None,2126-12-22 23:39:00,2126-12-23 02:30:00,Endotracheal tube
...,...,...,...,...,...
995,34786353,MMV/PSV,2153-08-13 20:00:00,2153-08-21 20:00:00,None
996,31544131,PCV+Assist,2177-08-25 00:00:00,2177-08-26 11:00:00,None
997,36931305,APV (cmv),2182-02-16 02:00:00,2182-03-03 11:00:00,None
998,35581526,MMV/PSV,2144-10-11 00:00:00,2144-10-14 05:00:00,None


Note:
- type is the ventilator mode
- value will be the oxygen delivery device

## Catheter-Associated Urinary Tract Infection (CAUTI)

In [15]:
cauti = """
-- mimiciv_derive.urine_output.sql
select
  stay_id,
  CASE itemid
    WHEN 226559 THEN 'Foley'
    WHEN 226561 THEN 'Condom Cath'
    WHEN 226584 THEN 'Ileoconduit'
    WHEN 226563 THEN 'Suprapubic'
    WHEN 226564 THEN 'R Nephrostomy'
    WHEN 226565 THEN 'L Nephrostomy'
    WHEN 226567 THEN 'Straight Cath'
    WHEN 226557 THEN 'R Ureteral Stent'
    WHEN 226558 THEN 'L Ureteral Stent'
    ELSE ''
    END as type,
  MIN(charttime) as starttime,
  MAX(charttime) as endtime,
  sum(urineoutput) as value
from
(
    select
    -- patient identifiers
    oe.stay_id,
    oe.itemid,
    oe.charttime,
    -- volumes associated with urine output ITEMIDs
    -- note we consider input of GU irrigant as a negative volume
    -- GU irrigant volume in usually has a corresponding volume out
    -- so the net is often 0, despite large irrigant volumes
    case
        when oe.itemid = 227488 and oe.value > 0 then -1*oe.value
        else oe.value
    end as urineoutput
    from `physionet-data.mimiciv_icu.outputevents` oe
    where itemid in
    (
    226559, -- Foley
    -- 226560, -- Void
    226561, -- Condom Cath
    226584, -- Ileoconduit
    226563, -- Suprapubic
    226564, -- R Nephrostomy
    226565, -- L Nephrostomy
    226567, -- Straight Cath
    226557, -- R Ureteral Stent
    226558 -- L Ureteral Stent
    -- 227488, -- GU Irrigant Volume In
    -- 227489  -- GU Irrigant/Urine Volume Out
    )
) uo
group by stay_id, itemid
HAVING
  MIN(charttime) != MAX(charttime)
LIMIT 1000
;
"""

In [16]:
run_query(cauti)

,stay_id,type,starttime,endtime,value
0,39765666,Foley,2189-06-27 09:08:00,2189-06-27 17:00:00,3900.0
1,31205490,Foley,2110-04-11 17:00:00,2110-04-12 20:00:00,2665.0
2,37510196,Foley,2131-01-11 08:00:00,2131-01-20 00:00:00,11708.0
3,39060235,Foley,2160-05-18 14:30:00,2160-05-19 16:00:00,2435.0
4,32358465,Foley,2131-03-10 00:00:00,2131-03-10 12:00:00,310.0
...,...,...,...,...,...
995,32506152,Foley,2122-04-10 23:00:00,2122-04-11 08:00:00,1390.0
996,35061245,Foley,2188-03-09 02:46:00,2188-03-09 17:00:00,1160.0
997,38528938,Foley,2184-05-03 15:03:00,2184-05-05 21:00:00,4090.0
998,31776655,Foley,2188-01-07 00:24:00,2188-01-08 10:00:00,3010.0


Note:
- type is the cathether type
- value is the urine output